Salimos de Madrid a Barcelona y Santiago de Compostela. Buscamos información de vuelos, alojamientos y actividades. Usamos dos apis y una web (se puede usar también dos webs y una ap). viaje 3 días.

1er paso elegir web que escrapear. No elegir sky scarapper ni trip advisor como web a escrapear

Paginas web candidatas a escrapear
    - https://feverup.com/es/barcelona?srsltid=AfmBOoodMcRV0dhtM9hp8YTlzX95bhg6Honac_KMdCe6eCAkp1Kcu0eW


In [29]:
!pip install pandas

In [30]:
# Importamos las librerías que previsiblemente necesitaremos
import pandas as pd
import numpy as np
# from src.support import obtener_df_coordenadas, buscar_lugares, obtener_servicios_municipios
# Uso de API's
# -----------------------------------------------------------------------
import requests

# Para incluir una barra de proceso en for loops
# -----------------------------------------------------------------------
from tqdm import tqdm


# Para introducir tiempo entre las llamadas
# -----------------------------------------------------------------------
from time import sleep


# Para trabajar con archivos dotenv y los tokens
# -----------------------------------------------------------------------
import os
import dotenv
dotenv.load_dotenv()

# Importamos BeautifulSoup
# ------------------------------------------------------------------------
from bs4 import BeautifulSoup 
#from src import soporte_beautifullsoup as sb

# librerias adicionales
from time import sleep
import random
import datetime

# Importar librerías para automatización de navegadores web con Selenium
# -----------------------------------------------------------------------
from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException # Excepciones comunes de selenium que nos podemos encontrar 